In [200]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython
import sklearn
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.deterministic import DeterministicProcess
from statsmodels.tsa.deterministic import CalendarFourier

import os

In [201]:
# path to the dataset in Kaggle's notebook
path ='input/store-sales-time-series-forecasting/'

## NN

### 1. Compute Moving Average of Oil Prices

In [202]:
# read oil price
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
#data_holiday = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
print(data_oil)
#print(data_holiday)
########################################################################################################################
# TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# Hint: check the documentation of .rolling() method of pandas.DataFrame
########################################################################################################################
data_oil['ma_oil'] = None # change 'None' to your answer
data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(5).mean() ### answer

# Create continguous moving average of oil prices
calendar_nn = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

########################################################################################################################
# TODO 1: merge two DataFrame instances (data_oil and calendar_nn) such that the merged instances has the same indexes
# as calendar_nn.
# TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
########################################################################################################################
calendar_nn = calendar_nn.merge(data_oil, how='left', left_index=True, right_index=True) ### answer
calendar_nn['ma_oil'].fillna(method='ffill', inplace=True) ### answer
calendar_nn['ma_oil'].fillna(method='ffill', inplace=True) ### answer

calendar_nn.head(15) # display some entries of calendar_nn

            dcoilwtico
date                  
2013-01-01         NaN
2013-01-02       93.14
2013-01-03       92.97
2013-01-04       93.12
2013-01-07       93.20
...                ...
2017-08-25       47.65
2017-08-28       46.40
2017-08-29       46.46
2017-08-30       45.96
2017-08-31       47.26

[1218 rows x 1 columns]


,dcoilwtico,ma_oil
2013-01-01,NaN,NaN
2013-01-02,93.14,NaN
2013-01-03,92.97,NaN
2013-01-04,93.12,NaN
2013-01-05,NaN,NaN
2013-01-06,NaN,NaN
2013-01-07,93.20,NaN
2013-01-08,93.21,93.128
2013-01-09,93.08,93.116
2013-01-10,93.81,93.284


### 2. Create Workday Feature

In [203]:
########################################################################################################################
# TODO: create a True/False feature calendar_nn['wd'] to indicate whether each date is a workday (Monday-Friday) or not.
# Hint: check documentation of pandas.DatetimeIndex.dayofweek
########################################################################################################################
#public sector paid
calendar_nn['pay'] = None
calendar_nn['pay'] = False
calendar_nn['dayind'] = calendar_nn.index.day
calendar_nn['wd'] = None # change 'None' to your answer
calendar_nn['dofw'] = calendar_nn.index.dayofweek ### answer
calendar_nn['wd'] = True ### answer
calendar_nn['month'] = calendar_nn.index.month ### answer
#months
calendar_nn['january']  = None
calendar_nn['february'] = None
calendar_nn['march'] = None
calendar_nn['april'] = None
calendar_nn['may'] = None
calendar_nn['june'] = None
calendar_nn['july'] = None
calendar_nn['august'] = None
calendar_nn['september'] = None
calendar_nn['october'] = None
calendar_nn['november'] = None
calendar_nn['december'] = None

calendar_nn['january']  = False
calendar_nn['february'] = False
calendar_nn['march'] = False
calendar_nn['april'] = False
calendar_nn['may'] = False
calendar_nn['june'] = False
calendar_nn['july'] = False
calendar_nn['august'] = False
calendar_nn['september'] = False
calendar_nn['october'] = False
calendar_nn['november'] = False
calendar_nn['december'] = False

#all days of week
calendar_nn['monday'] = None
calendar_nn['tuesday'] = None
calendar_nn['wednesday'] = None
calendar_nn['thursday'] = None
calendar_nn['friday'] = None
calendar_nn['saturday'] = None
calendar_nn['sunday'] = None

calendar_nn['monday'] = False
calendar_nn['tuesday'] = False
calendar_nn['wednesday'] = False
calendar_nn['thursday'] = False
calendar_nn['friday'] = False
calendar_nn['saturday'] = False
calendar_nn['sunday'] = False

#holidays
calendar_nn['bigholiday'] = None
calendar_nn['bigholiday'] = False

print(calendar_nn['dofw'
      ])
calendar_nn.loc[(calendar_nn.dofw < 4), 'wd'] = False ### answer
calendar_nn.loc[((calendar_nn.dayind > 14) & (calendar_nn.dayind < 16)) | (calendar_nn.index.is_month_end), 'pay'] = True
calendar_nn.loc[(calendar_nn.dofw == 0), 'monday'] = True ### answer
calendar_nn.loc[(calendar_nn.dofw == 1), 'tuesday'] = True ### answer
calendar_nn.loc[(calendar_nn.dofw == 2), 'wednesday'] = True ### answer
calendar_nn.loc[(calendar_nn.dofw == 3), 'thursday'] = True ### answer
calendar_nn.loc[(calendar_nn.dofw == 4), 'friday'] = True ### answer
calendar_nn.loc[(calendar_nn.dofw == 5), 'saturday'] = True ### answer
calendar_nn.loc[(calendar_nn.dofw == 6), 'sunday'] = True ### answer

print(calendar_nn)
calendar_nn.loc[(calendar_nn.month == 0), 'january'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 1), 'february'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 2), 'march'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 3), 'april'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 4), 'may'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 5), 'june'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 6), 'july'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 7), 'august'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 8), 'september'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 9), 'october'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 10), 'november'] = True ### answer
calendar_nn.loc[(calendar_nn.month == 11), 'december'] = True ### answer

calendar_nn.loc[(calendar_nn.month == 1) & (calendar_nn.dayind == 1) |
#             (calendar_nn.month == 12) & (calendar_nn.dayind == 31) |
#             (calendar_nn.month == 12) & (calendar_nn.dayind == 30) |
#             (calendar_nn.month == 12) & (calendar_nn.dayind == 29) |

             (calendar_nn.month == 5) & (calendar_nn.dayind == 30) |
#             (calendar_nn.month == 5) & (calendar_nn.dayind == 29) |
#             (calendar_nn.month == 5) & (calendar_nn.dayind == 28) |
#             (calendar_nn.month == 5) & (calendar_nn.dayind == 27) |

             (calendar_nn.month == 5) & (calendar_nn.dayind == 24) |
#             (calendar_nn.month == 5) & (calendar_nn.dayind == 23) |
#             (calendar_nn.month == 5) & (calendar_nn.dayind == 22) |
#             (calendar_nn.month == 5) & (calendar_nn.dayind == 21) |

             (calendar_nn.month == 7) & (calendar_nn.dayind == 24) |
#             (calendar_nn.month == 7) & (calendar_nn.dayind == 23) |
#             (calendar_nn.month == 7) & (calendar_nn.dayind == 22) |
#             (calendar_nn.month == 7) & (calendar_nn.dayind == 21) |

             (calendar_nn.month == 8) & (calendar_nn.dayind == 10) |
             (calendar_nn.month == 8) & (calendar_nn.dayind == 9) |
#             (calendar_nn.month == 8) & (calendar_nn.dayind == 8) |
#             (calendar_nn.month == 8) & (calendar_nn.dayind == 7) |

             (calendar_nn.month == 11) & (calendar_nn.dayind == 2) |
#             (calendar_nn.month == 11) & (calendar_nn.dayind == 1) |
#             (calendar_nn.month == 10) & (calendar_nn.dayind == 31) |
#             (calendar_nn.month == 10) & (calendar_nn.dayind == 30) |

             (calendar_nn.month == 11) & (calendar_nn.dayind == 25) |
#             (calendar_nn.month == 11) & (calendar_nn.dayind == 24) |
#             (calendar_nn.month == 11) & (calendar_nn.dayind == 23) |
#             (calendar_nn.month == 11) & (calendar_nn.dayind == 22) |

             (calendar_nn.month == 12) & (calendar_nn.dayind == 25) 
#             (calendar_nn.month == 12) & (calendar_nn.dayind == 24)
#             (calendar_nn.month == 12) & (calendar_nn.dayind == 23) |
#             (calendar_nn.month == 12) & (calendar_nn.dayind == 22)
            , 'bigholiday'] = True
calendar_nn.head(35) # display some entries of calendar_nn

2013-01-01    1
2013-01-02    2
2013-01-03    3
2013-01-04    4
2013-01-05    5
             ..
2017-08-27    6
2017-08-28    0
2017-08-29    1
2017-08-30    2
2017-08-31    3
Freq: D, Name: dofw, Length: 1704, dtype: int64
            dcoilwtico  ma_oil    pay  ...  saturday  sunday  bigholiday
2013-01-01         NaN     NaN  False  ...     False   False       False
2013-01-02       93.14     NaN  False  ...     False   False       False
2013-01-03       92.97     NaN  False  ...     False   False       False
2013-01-04       93.12     NaN  False  ...     False   False       False
2013-01-05         NaN     NaN  False  ...      True   False       False
...                ...     ...    ...  ...       ...     ...         ...
2017-08-27         NaN  47.676  False  ...     False    True       False
2017-08-28       46.40  47.478  False  ...     False   False       False
2017-08-29       46.46  47.240  False  ...     False   False       False
2017-08-30       45.96  46.742  False  ...    

,dcoilwtico,ma_oil,pay,dayind,wd,dofw,month,january,february,march,april,may,june,july,august,september,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday,bigholiday
2013-01-01,NaN,NaN,False,1,False,1,1,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True
2013-01-02,93.14,NaN,False,2,False,2,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
2013-01-03,92.97,NaN,False,3,False,3,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
2013-01-04,93.12,NaN,False,4,True,4,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2013-01-05,NaN,NaN,False,5,True,5,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
2013-01-06,NaN,NaN,False,6,True,6,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2013-01-07,93.20,NaN,False,7,False,0,1,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
2013-01-08,93.21,93.128,False,8,False,1,1,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
2013-01-09,93.08,93.116,False,9,False,2,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
2013-01-10,93.81,93.284,False,10,False,3,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False


### 3. Read Train and Test Data

In [204]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

df_train.head(15) # display some entries of the training data

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0
                     2013-01-06    2.0
                     2013-01-07    0.0
                     2013-01-08    2.0
                     2013-01-09    2.0
                     2013-01-10    2.0
                     2013-01-11    3.0
                     2013-01-12    2.0
                     2013-01-13    2.0
                     2013-01-14    2.0
                     2013-01-15    1.0

In [205]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

pd.set_option('display.max_columns', len(df_test.columns))

df_test.head(15) # display some entries of the testing data
df_test.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 28512 entries, ('1', 'AUTOMOTIVE', Period('2017-08-16', 'D')) to ('9', 'SEAFOOD', Period('2017-08-31', 'D'))
Empty DataFrame

In [206]:
# set the range of data used in training
sdate = '2017-04-01'
edate = '2017-08-15'

# we will train a model that takes feature of a date as input and predicts the sales for each store and family of goods on that date.
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]


########################################################################################################################
# TODO: create the trend feature X_nn: the value for sdate is 1, the value for the next day of sdate is 2, etc.
# Hint: check the documentation of DeterministicProcess, or this tutorial: https://www.kaggle.com/code/ryanholbrook/trend.
########################################################################################################################

X_nn = None # change 'None' to your answer
fourier = CalendarFourier(freq='W', order=4)
dp = DeterministicProcess(index=y.index,
                          constant=False,
                          order=1,
                          seasonal=False,
                          additional_terms=[fourier],
                          drop=True)
X_nn = dp.in_sample() ### answer
# Extentions
X_nn['oil']  = calendar_nn.loc[sdate:edate]['ma_oil'].values
X_nn['wd']   = calendar_nn.loc[sdate:edate]['wd'].values
X_nn['pay']  = calendar_nn.loc[sdate:edate]['pay'].values
X_nn['monday'] = calendar_nn.loc[sdate:edate]['monday'].values
X_nn['tuesday'] = calendar_nn.loc[sdate:edate]['tuesday'].values
X_nn['wednesday'] = calendar_nn.loc[sdate:edate]['wednesday'].values
X_nn['thursday'] = calendar_nn.loc[sdate:edate]['thursday'].values
X_nn['friday'] = calendar_nn.loc[sdate:edate]['friday'].values
X_nn['saturday'] = calendar_nn.loc[sdate:edate]['saturday'].values
X_nn['sunday'] = calendar_nn.loc[sdate:edate]['sunday'].values

X_nn['january'] = calendar_nn.loc[sdate:edate]['january'].values
X_nn['february'] = calendar_nn.loc[sdate:edate]['february'].values
X_nn['march'] = calendar_nn.loc[sdate:edate]['march'].values
X_nn['april'] = calendar_nn.loc[sdate:edate]['april'].values
X_nn['may'] = calendar_nn.loc[sdate:edate]['may'].values
X_nn['june'] = calendar_nn.loc[sdate:edate]['june'].values
X_nn['july'] = calendar_nn.loc[sdate:edate]['july'].values
X_nn['august'] = calendar_nn.loc[sdate:edate]['august'].values
X_nn['september'] = calendar_nn.loc[sdate:edate]['september'].values
X_nn['october'] = calendar_nn.loc[sdate:edate]['october'].values
X_nn['november'] = calendar_nn.loc[sdate:edate]['november'].values
X_nn['december'] = calendar_nn.loc[sdate:edate]['december'].values

X_nn['bigholiday'] = calendar_nn.loc[sdate:edate]['bigholiday'].values

#df_test = df_test.set_index('date')

# Use the pivot method to reshape the dataframe and create a new column for each str_nbr value
#df_test = df_test.pivot(columns='str_nbr')

# Rename the columns to include the str_nbr value
#df_test.columns = [f'str_{c[1]}' for c in df_test.columns]

# Merge the reshaped dataframe with X_nn
#X_nn = pd.concat([X_nn, df_test], axis=1)
X_nn.head(15)

,trend,oil,wd,pay,monday,tuesday,wednesday,thursday,friday,saturday,sunday,january,february,march,april,may,june,july,august,september,october,november,december,bigholiday
date,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-01,1.0,49.138,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-02,2.0,49.138,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-03,3.0,49.784,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-04,4.0,50.310,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-05,5.0,50.644,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-06,6.0,50.922,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-07,7.0,51.264,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-08,8.0,51.264,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-09,9.0,51.264,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False


### Train NN Model

In [207]:
#model = Ridge(fit_intercept=True, solver='auto', alpha=0.5, normalize=True)
model = MLPRegressor(hidden_layer_sizes=(64, 64), learning_rate_init=0.07, max_iter = 5000)
new_model = MultiOutputRegressor(model)
new_model.fit(X_nn, y)

#partial fit
#for i in range(50):
#    new_model.partial_fit(X_nn, y)
    #scores = []
    #scores.append(new_model.score(X_nn, y))
#    print("Iteration {}, Test score = {}".format(i+1, new_model.score(X_nn, y)))
y_pred = pd.DataFrame(new_model.predict(X_nn), index=X_nn.index, columns=y.columns)

#get loss scores
print(new_model.score(X_nn, y))

# Results on the training set
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

y_target['sales_pred'] = y_pred['sales'].clip(0.) # Sales should be >= 0

########################################################################################################################
# TODO: show the training loss for each type of product.
# Hint: check the documentation of DataFrame.groupby() and GroupBy.apply().
########################################################################################################################
NN_result = y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])) ### answer
# print(y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])).sum())

0.2987234729040762


# RF

## Feature Engineering

In [208]:
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

## Oil

In [209]:
df_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
df_oil['ma_oil'] = df_oil['dcoilwtico'].rolling(7).mean()
calendar = calendar.merge(df_oil, how='left', left_index=True, right_index=True)
calendar['ma_oil'].fillna(method='bfill', inplace=True)
calendar.head()

,dcoilwtico,ma_oil
2013-01-01,NaN,93.218571
2013-01-02,93.14,93.218571
2013-01-03,92.97,93.218571
2013-01-04,93.12,93.218571
2013-01-05,NaN,93.218571


## Day of week

In [210]:
calendar['dow'] = calendar.index.dayofweek
calendar.head()

,dcoilwtico,ma_oil,dow
2013-01-01,NaN,93.218571,1
2013-01-02,93.14,93.218571,2
2013-01-03,92.97,93.218571,3
2013-01-04,93.12,93.218571,4
2013-01-05,NaN,93.218571,5


## Hoilday

In [211]:
holidays = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True)
holidays = holidays.set_index('date').sort_index()
holidays = holidays[holidays.locale == 'National'] # National level only for simplicity
#holidays = holidays.groupby(holidays.index).first() # Keep one event only
holidays.head()

,type,locale,locale_name,description,transferred
date,,,,,
2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False


## Work Day

In [212]:
calendar['wd'] = True
calendar.loc[calendar.dow > 4, 'wd'] = False
calendar = calendar.merge(holidays, how='left', left_index=True, right_index=True)
calendar.loc[calendar.type == 'Bridge'  , 'wd'] = False
calendar.loc[calendar.type == 'Work Day', 'wd'] = True
calendar.loc[calendar.type == 'Transfer', 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True ), 'wd'] = True
calendar.head()

,dcoilwtico,ma_oil,dow,wd,type,locale,locale_name,description,transferred
2013-01-01,NaN,93.218571,1,False,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-02,93.14,93.218571,2,True,NaN,NaN,NaN,NaN,NaN
2013-01-03,92.97,93.218571,3,True,NaN,NaN,NaN,NaN,NaN
2013-01-04,93.12,93.218571,4,True,NaN,NaN,NaN,NaN,NaN
2013-01-05,NaN,93.218571,5,True,Work Day,National,Ecuador,Recupero puente Navidad,False


## Drop 12/25 which training set exclude

In [213]:
calendar = calendar.reset_index()
calendar.drop_duplicates(subset=['index'], keep='last', inplace=True, ignore_index=True)
calendar = calendar.set_index('index')
calendar = calendar.drop(index=['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'])
print(calendar.shape)

(1700, 9)


## Training

In [214]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()
df_train.head()

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0

### Label (y)

In [215]:
sdate = '2017-04-01'
edate = '2017-08-15'
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]
print(y.shape)
y.head()

(137, 1782)


sales            ...                                      
store_nbr           1            ...                          9           
family     AUTOMOTIVE BABY CARE  ... SCHOOL AND OFFICE SUPPLIES    SEAFOOD
date                             ...                                      
2017-04-01        9.0       0.0  ...                        1.0  40.446999
2017-04-02        4.0       0.0  ...                        3.0  15.000000
2017-04-03       11.0       0.0  ...                        2.0  16.000000
2017-04-04        3.0       0.0  ...                        2.0  15.000000
2017-04-05        5.0       0.0  ...                        1.0  13.373000

[5 rows x 1782 columns]

### Data point (X)

In [216]:
fourier = CalendarFourier(freq='W', order=4)
dp = DeterministicProcess(index=y.index,
                          constant=False,
                          order=1,
                          seasonal=False,
                          additional_terms=[fourier],
                          drop=True)
X = dp.in_sample()

# Extentions
X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values
X['dow'] = calendar.loc[sdate:edate]['dow'].values
X['wd']   = calendar.loc[sdate:edate]['wd'].values
X['type'] = calendar.loc[sdate:edate]['type'].values
X = pd.get_dummies(X, columns=['dow'], drop_first=True)
X = pd.get_dummies(X, columns=['type'], drop_first=False)
print(X.shape)
X.head()


(137, 13)


,trend,oil,wd,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,type_Additional,type_Event,type_Holiday,type_Transfer
date,,,,,,,,,,,,,
2017-04-01,1.0,49.034286,False,0,0,0,0,1,0,0,0,0,0
2017-04-02,2.0,49.034286,False,0,0,0,0,0,1,0,0,0,0
2017-04-03,3.0,49.034286,True,0,0,0,0,0,0,0,0,0,0
2017-04-04,4.0,49.561429,True,1,0,0,0,0,0,0,0,0,0
2017-04-05,5.0,50.150000,True,0,1,0,0,0,0,0,0,0,0


### Train

In [217]:
# model = LinearRegression()
# model.fit(X,y)
model_rf = RandomForestRegressor(n_estimators=250,random_state=2022, verbose=0)
model_rf.fit(X, y)

RandomForestRegressor(n_estimators=250, random_state=2022)

In [218]:
y_pred = pd.DataFrame(model_rf.predict(X), index=X.index, columns=y.columns)
y_pred=y_pred.stack(['store_nbr', 'family']).reset_index()
y_pred.head()

,date,store_nbr,family,sales
0,2017-04-01,1,AUTOMOTIVE,7.464
1,2017-04-01,1,BABY CARE,0.000
2,2017-04-01,1,BEAUTY,1.296
3,2017-04-01,1,BEVERAGES,2782.444
4,2017-04-01,1,BOOKS,0.016


In [219]:
y_target=y.stack(['store_nbr', 'family']).reset_index().copy()
y_target['sales_pred'] = y_pred['sales'].clip(0.) # Sales should be >= 0
RF_result = y_target.groupby('family').apply(lambda x: np.sqrt(mean_squared_log_error(x['sales'], x['sales_pred'])))

# Retrain

## Comapre two model

In [220]:
NN_list = []
RF_list = []
for i in NN_result.index:
    if NN_result.loc[i]+0.1 <= RF_result.loc[i]:
        NN_list.append(i)
    else:
        RF_list.append(i)
print(NN_list)
print(RF_list)

['BEVERAGES', 'BREAD/BAKERY', 'CLEANING', 'DAIRY', 'DELI', 'FROZEN FOODS', 'GROCERY I', 'HOME CARE', 'LIQUOR,WINE,BEER', 'MEATS', 'PERSONAL CARE', 'POULTRY', 'PREPARED FOODS', 'PRODUCE']
['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BOOKS', 'CELEBRATION', 'EGGS', 'GROCERY II', 'HARDWARE', 'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE', 'MAGAZINES', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'SCHOOL AND OFFICE SUPPLIES', 'SEAFOOD']


## Data set

In [221]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')

In [222]:
NN_train = pd.DataFrame()
for item in NN_list:
    df = df_train[df_train['family'] == item]
    NN_train = pd.concat([NN_train, df])
NN_train = NN_train.set_index(['store_nbr', 'family', 'date']).sort_index()

RF_train = pd.DataFrame()
for item in RF_list:
    df = df_train[df_train['family'] == item]
    RF_train = pd.concat([RF_train, df])
RF_train = RF_train.set_index(['store_nbr', 'family', 'date']).sort_index()

In [223]:
sdate = '2017-04-01'
edate = '2017-08-15'
y_nn = NN_train.unstack(['store_nbr', 'family']).loc[sdate:edate]
y_rf = RF_train.unstack(['store_nbr', 'family']).loc[sdate:edate]

## Train Model

In [224]:
model = MLPRegressor(hidden_layer_sizes=(64, 64), learning_rate_init=0.07, max_iter = 5000)
new_model = MultiOutputRegressor(model)
new_model.fit(X_nn, y_nn)

MultiOutputRegressor(estimator=MLPRegressor(hidden_layer_sizes=(64, 64),
                                            learning_rate_init=0.07,
                                            max_iter=5000))

In [225]:
model_rf = RandomForestRegressor(n_estimators=250,random_state=2022, verbose=0)
model_rf.fit(X, y_rf)

RandomForestRegressor(n_estimators=250, random_state=2022)

# Test

In [226]:
# Test predictions

stest = '2017-08-16'
etest = '2017-08-31'

########################################################################################################################
# TODO: create the feature matrix of test data.
# Hint: check the documentation of DeterministicProcess.
########################################################################################################################
X_test = None # change 'None' to your answer
X_test = dp.out_of_sample(steps=16) ### answer

# Extentions

X_test['oil']  = calendar_nn.loc[stest:etest]['ma_oil'].values ### answer

X_test['wd']   = calendar_nn.loc[stest:etest]['wd'].values ### answer
X_test['pay']  = calendar_nn.loc[stest:etest]['pay'].values
X_test['monday'] = calendar_nn.loc[stest:etest]['monday'].values
X_test['tuesday'] = calendar_nn.loc[stest:etest]['tuesday'].values
X_test['wednesday'] = calendar_nn.loc[stest:etest]['wednesday'].values
X_test['thursday'] = calendar_nn.loc[stest:etest]['thursday'].values
X_test['friday'] = calendar_nn.loc[stest:etest]['friday'].values
X_test['saturday'] = calendar_nn.loc[stest:etest]['saturday'].values
X_test['sunday'] = calendar_nn.loc[stest:etest]['sunday'].values

X_test['january'] = calendar_nn.loc[stest:etest]['january'].values
X_test['february'] = calendar_nn.loc[stest:etest]['february'].values
X_test['march'] = calendar_nn.loc[stest:etest]['march'].values
X_test['april'] = calendar_nn.loc[stest:etest]['april'].values
X_test['may'] = calendar_nn.loc[stest:etest]['may'].values
X_test['june'] = calendar_nn.loc[stest:etest]['june'].values
X_test['july'] = calendar_nn.loc[stest:etest]['july'].values
X_test['august'] = calendar_nn.loc[stest:etest]['august'].values
X_test['september'] = calendar_nn.loc[stest:etest]['september'].values
X_test['october'] = calendar_nn.loc[stest:etest]['october'].values
X_test['november'] = calendar_nn.loc[stest:etest]['november'].values
X_test['december'] = calendar_nn.loc[stest:etest]['december'].values

X_test['bigholiday'] = calendar_nn.loc[stest:etest]['bigholiday'].values

sales_pred_nn = pd.DataFrame(new_model.predict(X_test), index=X_test.index, columns=y_nn.columns)
# sales_pred_= sales_pred.stack(['store_nbr', 'family'])

sales_pred_nn[sales_pred_nn < 0] = 0. # Sales should be >= 0

In [227]:
X_test_rf = dp.out_of_sample(steps=16)

# Extentions
X_test_rf['oil']  = calendar.loc[stest:etest]['ma_oil'].values
X_test_rf['dow'] = calendar.loc[stest:etest]['dow'].values
X_test_rf['wd']   = calendar.loc[stest:etest]['wd'].values
X_test_rf['type'] = calendar.loc[stest:etest]['type'].values
X_test_rf = pd.get_dummies(X_test_rf, columns=['dow'], drop_first=True)
X_test_rf = pd.get_dummies(X_test_rf, columns=['type'], drop_first=False)

# No national level events in this period
X_test_rf[['type_Additional', 'type_Event', 'type_Holiday', 'type_Transfer']] = 0
X_test_rf.index.names = ['date']

sales_pred_rf = pd.DataFrame(model_rf.predict(X_test_rf), index=X_test_rf.index, columns=y_rf.columns)
# sales_pred_rf = sales_pred_rf.stack(['store_nbr', 'family']).reset_index()
sales_pred_rf[sales_pred_rf < 0] = 0 # Sales should be >= 0

In [228]:
pred_final = sales_pred_rf.merge(sales_pred_nn, left_index=True, right_index=True)
pred_final = pred_final.stack(['store_nbr', 'family']).reset_index()

In [229]:
# Create submission
df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
df_sub.sales = pred_final['sales'].values
df_sub.to_csv('submission.csv', index=True)